<img src="https://drive.google.com/uc?export=view&id=1x-QAgitB-S5rxGGDqxsJ299ZQTfYtOhb" width=180, align="center"/>

Master's degree in Intelligent Systems

Subject: 11754 - Deep Learning

Year: 2022-2023

Professor: Miguel Ángel Calafat Torrens


# LAB 4

In this lab you have to deliver two files.

* On the one hand you must deliver the file helper_PR4.py adding the functions described in problem 1.

* On the other hand, you must deliver the notebook `LSS4.2.ipynb` suitably modified according to the instructions of problem 2. In it you can modify as many cells as you consider appropriate to carry out the proposed objective.

You don't have to deliver this notebook (`LAB-4.ipynb`).

## Problem 1

In the `LSS4.1.ipynb` notebook you have trained and validated a model. The validation is to realize if overfitting is taking place; however, the true test for the model are the results with the test dataset.

In this exercise you will create two functions and leave them saved in the script called `helper_PR4.py`.

The first function will be the `test_pass()` function. This function is, for the test data set, the equivalent of `train_pass()` for the training data set, or the equivalent of `valid_pass()` for the validation data set. Both functions have been defined in the `LSS4.1` notebook.

Following is the signature of the function, the description of the operation and the names of the output parameters.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# As with train_pass and valid_pass, we define a function for a single test
# step. This function returns the losses and also the output of the model.

# Remember that the output of the model will have dimensions of b x 10
def test_pass(data, target, model, criterion):
    """
    Evaluates a given model on a single batch of data.

    Args:
        data (torch.Tensor): Input data tensor.
        target (torch.Tensor): Target tensor.
        model (torch.nn.Module): PyTorch model to be evaluated.
        criterion (torch.nn.Module): Loss function.

    Returns:
        Tuple containing the loss value and output tensor.
    """
    # set model to evaluation mode
    model.eval()

    # move data and target to device
    data, target = data.to(DEVICE), target.to(DEVICE)

    # perform forward pass
    with torch.no_grad():
        output = model(data)
        loss = criterion(output, target)

    # return loss value and output tensor
    return loss.item(), output


Once the previous function is finished and saved in `helper_PR4.py` you must prepare another function. In this case it is the `do_test()` function. This is a function in which one pass will be made through the entire test dataset and the accuracy, unit losses and model output will be computed.

In [ ]:
def do_test(model, loaders, criterion):
    """
    Test the performance of the given model on the test set.
    
    Args:
        model: A PyTorch model.
        loaders: A dictionary with 'test' DataLoader.
        criterion: A PyTorch loss function.

    Returns:
        accuracy: A float representing the accuracy on the test set.
        test_loss: A float representing the average test loss.
        all_outputs: A tensor containing the model outputs on the test set
    """

    # Set the model to evaluation mode
    model.eval()

    # move model to device
    model.to(DEVICE)

    # create DataLoader for test set
    test_loader = loaders['test']

    # initialize variables
    test_loss = 0.0
    num_correct = 0
    all_outputs = torch.tensor([]).to(DEVICE)

    # evaluate model on test set
    for data, target in test_loader:

        # perform test pass
        loss, output = test_pass(data, target, model, criterion)

        # update loss and accuracy
        test_loss += loss.item() * data.size(0)
        num_correct += torch.sum(torch.argmax(output, dim=1) == target).item()

        # concatenate all outputs
        all_outputs = torch.cat((all_outputs, output), dim=0)

    # calculate average test loss and accuracy
    test_loss /= len(test_loader.dataset)
    accuracy = num_correct / len(test_loader.dataset)

    # return accuracy, test loss and all outputs
    return accuracy, test_loss, all_outputs

## Problem 2

The second problem is to completely re-run the `LSS4.2` notebook by introducing the following change: you have to define your own CustomDataset class from the PyTorch Dataset class.

For this you can use the following model. The class must have at least the properties and methods defined below.

The input `im_paths` should be a list with the path of every image in the dataset. In the `__init__` function you should extract all the labels of the images. The label of an image is a string with the letters of the filename untill the first digit is found. For example, `shine123.jpg` and `shine21.jpg` will have the label `shine`; `rain215.jpg` will have the label `rain`, and so on.

The labels should be recorded as integers for the training, so in `self.labels` you should store a list of labels in integer format (you can choose which number equals which class; a usual option is sorted in alphabetical order)

In [ ]:
from torch.utils.data import Dataset
from typing import List
from PIL import Image
import torch

# Define new class Dataset specific for this project
class CustomDataset(Dataset):
    def __init__(self, im_paths, transform=None):
        self.image_paths = im_paths  # List of paths as strings
        self.transform = transform  # Transformations
        self.classes = sorted(set([self.get_label(path) for path in im_paths]))  # Unique labels as strings
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}  # Dictionary mapping classes to indexes. For
                                  # example: {'cloudy':0, `rain`:1, ...}
        self.labels = [self.class_to_idx[self.get_label(path)] for path in im_paths]
        # List of integers with labels corresponding to the
                            # list of paths (image_paths)


    def __len__(self):
        # Don't do anything in this function. It is provided for you
        return len(self.labels)


    def __getitem__(self, idx):
        # Do all the necessary treatment of data. Remember that when you apply
        # next() to the daloader, it will call this function. So you have to
        # do all the necessary steps to deliver the image and the label.

        # You don't have to worry about the batch; it's controlled by the
        # dataloader. You just have to return an image and label pointed by
        # an index (idx)
        
        path = self.image_paths[idx]
        image = Image.open(path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, torch.tensor(label)

    def get_label(self, path):
        """
        Returns the label of an image, which is the string of characters
        in the filename until the first digit is found.
        """
        filename = path.split('/')[-1]  # get the filename from the path
        label = ''
        for char in filename:
            if char.isdigit():
                break
            label += char
        return label